In [1]:
from tensorflow import keras # Do this

# Keras Input 

## Input is NOT a Layer

Note that ```keras.Input``` is NOT in ```keras.layer``` package as it is not a Layer but a **Tensor**,

* [Input Object](https://keras.io/api/layers/core_layers/input/)

> Input() is used to instantiate a Keras **tensor**.



## Input Shape

Because the preprocessing normalization layer is the first layer to take inputs, it requires ```input_shape```.

* [Keras input explanation: input_shape, units, batch_size, dim, etc](https://stackoverflow.com/a/44748370/4281353)

> Example: if you have 30 images of 50x50 pixels in RGB (3 channels), the shape of your input data is (30,50,50,3). 
> 
> Keras demands the input shape at the first layer only, and Keras ignores the first dimension of the input data, which is the batch size. Your model should be able to deal with any batch size, so you define only the other dimensions:
> ```
> #regardless of how many images I have, each image has this shape   
> input_shape = (50,50,3)
> ```


# Regularizer

* [What is the difference between kernel, bias, and activity regulizers](https://stats.stackexchange.com/a/383326/105137)

> With a neural network regression equation ```𝑦=𝑊𝑥+𝑏```
> * Kernel Regularizer: Tries to reduce the weights 𝑊 (excluding bias).
> * Bias Regularizer: Tries to reduce the bias 𝑏
> * Activity Regularizer: Tries to reduce the layer's output ```𝑦``` thus will reduce the weights and adjust bias so ```𝑊𝑥+𝑏``` is smallest.
> 
> Usually, if you have no prior on the distribution that you wish to model, you would only use the kernel regularizer, since a large enough network can still model your function even if the regularization on the weights are big.
> 
> If you want the output function to pass through (or have an intercept closer to) the origin, you can use the bias regularizer.

---
# Categorical Label vs Sparce Categorical Label

## Sparse Categorical Label

A label is a single index of a class.

```
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print(y_train[:5])
-----
[[6]
 [9]
 [9]
 [4]
 [1]]
```

Need to use ```sparce categorical``` for the loss calculation and metrics to evaluate.

* [tf.keras.metrics.sparse_categorical_crossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/sparse_categorical_crossentropy)


```
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.sparse_categorical_crossentropy, 
    # metrics=['accuracy']   # <--- 'TF/Keras auto detect categorical or sparse category with 'accuracy' string
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)
```

## Categorical Label

A label is one-hot-encoded. Need to use ```categorical``` for the loss calculation and metrics to evaluate.


* [tf.keras.metrics.categorical_crossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/categorical_crossentropy)

```
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.categorical_crossentropy, 
    # metrics=['accuracy']
    metrics=[tf.keras.metrics.CategoricalAccuracy()]
)
```

---
# Evaluation Metrics

During the training, TF/Keras calculates the evaluation metrics e.g. accuracy, precision, auc, etc.

```
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.sparse_categorical_crossentropy, 
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]  # <--- metrics Keras calculates during the training
)
```

**String metric** identifier can be used instead of the metrics instance.

```
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.sparse_categorical_crossentropy, 
    # metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    metrics=['sparse_categorical_accuracy']
)
```

For **convenience**, we can specify a string ```'accuracy'``` which TF/Keras auto-detect the correct accuracy metric.

```
model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.sparse_categorical_crossentropy, 
    # metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    metrics=['accuracy']  <--- 'TF/Keras auto detect categorical or sparse category with 'accuracy' string
)
```



## List of Metrics

See [Module: tf.keras.metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics) for the available metrices.


## Availability of Metrics

Metrics e.g. Recall, Prcision, AUC are only for **Binary** categorical labels only.

Example of the error using ```precision``` for multi categorical label. 

```
print(y_train[:5])
-----
[[6]
 [9]
 [9]
 [4]
 [1]]

model.compile(
    optimizer='adam', 
    loss=tf.keras.losses.sparse_categorical_crossentropy, 
    # metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    metrics=[tf.keras.metrics.Precision()]
)
model.fit(X=x_train, y_train)
-----
ValueError: Shapes (32, 10) and (32, 1) are incompatible  # <--- expected binary 1 class, got 10 classes.
```


---
# Monitoring Metric

TF/Keras callbacks e.g. ```EarlyStopping``` monitor the validation metrics during the trainig, 

* [TensorFlow/Keras where are 'loss', 'mean_absolute_error', 'val_loss', 'val_mean_absolute_error' defined?](https://stackoverflow.com/questions/75479364)

> Specify the prefix ```val_```, with the metric computed on validation set. [Available metrics](https://keras.io/api/metrics/) to find the metric names (except ```loss```).

```
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # <--- e.g. val_accuracy, val_auc
    mode='min',
    patience=5,
    verbose=1,
    restore_best_weights=True,
)

history = model.fit(
    x, 
    y,
    shuffle=True,
    batch_size=batch_size,
    epochs=number_of_epochs,
    validation_split=validation_split,
    use_multiprocessing=use_multiprocessing,
    workers=workers,
    verbose=verbosity,
    callbacks=[
        tensorboard_callback,
        earlystop_callback
    ]
)
```


---
# Monitoring Mode

* [ModelCheckpoint](https://keras.io/api/callbacks/model_checkpoint/)

> * mode: one of {'auto', 'min', 'max'}.  
> 
> Based on either **the maximization or the minimization of the monitored quantity**. For ```val_acc```, this should be **max**, for ```val_loss``` this should be **min**, etc. In ```auto``` mode, the mode is set to **max** if the quantities monitored are ```'acc'``` or start with ```'fmeasure'``` and are set to ```min``` for the rest of the quantities.

```
earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',          # <--- validation loss should be minimizing, hence 'min'
    patience=5,
    verbose=1,
    restore_best_weights=True,
)
```

---
# Backend

* [Backend utilities](https://keras.io/api/utils/backend_utils/)
* [tf.keras.backend.set_image_data_format](https://www.tensorflow.org/api_docs/python/tf/keras/backend/set_image_data_format)

```
tf.keras.backend.set_image_data_format('channels_first')
tf.keras.backend.set_image_data_format('channels_last')
```

* [regarding set_image_dim_ordering('tf') in keras](https://stackoverflow.com/a/66460620/4281353)

> Keras is Deep Learning framework which can uses from 'TensorFlow' , 'Theano' and 'CNTK' as a backend. Every backend has its own preferences to use the channel ordering
> * Theano: channel first order  
> ```K.set_image_data_format('channels_first')```
> * Tensorflow: channel last order  
> ```K.set_image_data_format('channels_last')```
> * CNTK: channel last order
> 
> Note: channel order in your "data shape" should be equal to channel order in the "backend". for example if you are using Tensorflow as backend, then the input shape of your data should be channel last order(for RGB image: (rows, columns, channel))
> 

In [2]:
from keras import backend as K
K.backend()

'tensorflow'

In [3]:
K.image_data_format()

'channels_last'